
**Scraping Toronto Neighborhoods Table from Wikipedia**

In [129]:
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df=pd.read_html(url, header=0)[0]

df.head()




,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [130]:
df.columns

Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')

In [131]:
df.rename(columns = {'Postal Code':'PostalCode'}, inplace = True) 
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


clean and display the top 10 rows

In [132]:
#Data Cleaning 

df = df[df.Borough != 'Not assigned']
df = df[df.Borough!=0]
df.reset_index(drop=True, inplace=True)
i =0

for i in range(0, df.shape[0]):
  if df.iloc[i][2] == 'Not assigned':
    df.loc[i][2] = df.iloc[i][1]
    i = i+1
df = df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [133]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.PostalCode != empty ) & (df.Borough != empty) & (df.Neighbourhood != empty)]

In [134]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
    len(df['Borough'].unique()),df.shape[0]))

The dataframe has 10 boroughs and 103 neighbourhoods.


In [135]:

def neighbourhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighbourhood'].tolist()))
                    
grp = df.groupby(['PostalCode', 'Borough'])
df2 = grp.apply(neighbourhood_list).reset_index(name='Neighbourhood')

In [136]:
print(df2.shape)
df2.head(11)

(103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [137]:
print('The DataFrame shape is', df2.shape)


The DataFrame shape is (103, 3)


The dataframe has 103 Postal codes but it has 212 rows, because each Postal code can present more than one neighborhood (210 in total). Therefore, the dataframe should be group by the Postal code, ending with a dataframe with 103 rows.

**I’m using “https://cocl.us/Geospatial_data” csv file to get all the geographical coordinates of the neighborhoods.**

In [138]:
import io 
import requests

url = 'https://cocl.us/Geospatial_data'

lat_long =requests.get(url).text
lat_long_df = pd.read_csv(io.StringIO(lat_long))
lat_long_df.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [139]:
#rename columns
lat_long_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)


In [140]:
#merge the tables 
df = pd.merge(df, lat_long_df, on = 'PostalCode')
df.head(11)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
